<a href="https://colab.research.google.com/github/OffShur3/NixosGenerations/blob/main/Delivery%20note%20with%20OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nota de entrega con OCR
En este programa nos genera notas de entrega para todos los consumos pendientes de las instalaciones, tiene un modo de reconocimiento de texto dentro de las fotos para poder detectar en caso de que haya algun serie mal escrito

## Importacion de librerias
Esto es lo necesario para que el programa funcione, sin contar con el reconocimiento de OCR que aparece luego en caso de que se necesite

In [2]:
# Importacion de librerias
import csv
import json
import requests
import os
import sys
import re
import pprint
import cv2
from time import sleep
from datetime import date, datetime
from urllib.parse import urlencode
from collections import defaultdict
from requests.adapters import HTTPAdapter
from requests.utils import dict_from_cookiejar

from google.colab import userdata

### **Variables modificables**

* `planilla_anual`: URL de la planilla de Google Sheets que contiene los datos de consumo. Es un recurso vital para el funcionamiento del script.
* **APIs del ERP:** Estas son las claves (`API_KEY` y `API_SECRET`) que se generan en Frappe. Son esenciales para la autenticación y el intercambio de datos con el servidor.
* `FRAPPE_API`: Es el dominio o la URL donde está alojado el servidor de Frappe. El script obtiene esta URL de una celda específica de otra planilla de Google Sheets.
* `gpu_o_cpu`: Variable para seleccionar si el proceso de OCR se ejecutará en una GPU o en una CPU. Por defecto, está configurada para usar la CPU.
* **Otras variables:** El resto de las variables (`archivo_crudo`, `archivo_Series`, etc.) son nombres de archivos temporales que no necesitan ser modificados. Los prefijos válidos (`prefijos_validos`) se utilizan para validar los números de serie detectados por el OCR.

In [3]:
#ocr con gpu o cpu?
gpu_o_cpu = "C" #input("Para hacer el OCR usamos [G]pu o [C]pu?")

# === ~ Variables ===

#crudo
planilla_anual = userdata.get('planilla_anual')
# URL Apps Script del Forms
FormsAppscript = userdata.get('FormsAppscript')

# === 🔐 API ERP ===
API_KEY = userdata.get('API_KEY')
API_SECRET = userdata.get('API_SECRET')
FRAPPE_API_crudo = requests.get("https://docs.google.com/spreadsheets/d/e/2PACX-1vRbnN152KxnzZqA3GA0L0LQJCF9uGRNIXsydi_FB1EveJ_aVSD_5rjHtTcfoGyctN8ZPou5R7Eg-QQ8/pub?gid=1381182595&single=true&output=csv").text.splitlines()[1]
FRAPPE_API = FRAPPE_API_crudo+"/api/resource"


# Nombre de los archivos a generar
archivo_crudo = "crudo.csv" #para el crudo de las OT
archivo_Series = "series_frappe.csv" #para el listado de series de frappe
archivo_productos = "productos.csv" #para el listado de productos
archivo_ONT = "columnas_extraidasONT.csv" #columnas de extraccion de datos ONT
archivo_DECO = "columnas_extraidasDeco.csv" #columnas de extraccion de datos Decos
archivo_consumibles = "reporte_consumibles.csv" #cables usados
path_viejos="url_serieViejo.csv"
path_nuevos="series_validas_detectadas.csv"
prefijos_validos = ['4857', 'HP40', 'HP44', 'FZ32', '534D', 'ALCL']

## Nota de entrega

In [1]:
def install_other_dependencies():
    import subprocess

    # --- 1. Verificación del entorno Colab ---
    # El módulo google.colab solo existe en ese entorno
    try:
        import google.colab
        IN_COLAB = True
        print("Detectado: Google Colab. Se ejecutarán comandos de Colab.")
    except ImportError:
        IN_COLAB = False
        print("Detectado: Entorno local.")

    # --- 2. Lógica condicional ---
    if IN_COLAB:
        # Código específico para Google Colab
        print("\n--- Instalando dependencias con comandos de Colab ---")
        !pip install pytesseract
        !apt-get install -y tesseract-ocr libtesseract-dev
        !pip install paddleocr
        # Puedes añadir la lógica para PaddlePaddle aquí
        # !pip install paddlepaddle-gpu o !pip install paddlepaddle
    else:
        # Código específico para un entorno local
        try:
            # --- Si es Linux, ejecuta los comandos con subprocess ---
            print("\n--- Detectado: Linux. Instalando dependencias con apt y pip ---")

            # Comandos de apt-get
            print("Ejecutando apt-get update...")
            subprocess.run(["sudo", "apt-get", "update"], check=True)

            print("Ejecutando apt-get install tesseract-ocr...")
            subprocess.run(["sudo", "apt-get", "install", "-y", "tesseract-ocr", "libtesseract-dev"], check=True)

            # Comandos de pip
            print("Instalando librerías de Python con pip...")
            subprocess.run(["pip", "install", "pytesseract"], check=True)
            if gpu_o_cpu == "G":
                subprocess.run(["pip", "install", "paddlepaddle-gpu"], check=True)
            else:
                subprocess.run(["pip", "install", "paddlepaddle"], check=True)

        except:
            print("\n--- Sistema operativo no compatible con este script de instalación. ---")
            print("Por favor, instala las dependencias manualmente.")

### **Funciones de limpieza y estado**

* `borrar_archivos_csv()`: Esta función elimina los archivos CSV temporales que se crean durante la ejecución del script. Su propósito es mantener el entorno limpio y evitar que se usen datos de ejecuciones anteriores.

* `hay_que_revisar(only_boolean=False)`: Verifica si hay equipos pendientes de transferencia o que no fueron recibidos. Lee los archivos `ONT.csv` y `DECO.csv` para determinar su estado. La función puede devolver un valor booleano (`True` o `False`) si se especifica, o imprimir mensajes detallados sobre los equipos que requieren atención.

In [47]:
def borrar_archivos_csv():
    for archivo in [archivo_crudo, archivo_Series, archivo_productos, archivo_ONT, archivo_DECO, archivo_consumibles]:
        if os.path.exists(archivo):
            os.remove(archivo)
            print(f"🧹 Archivo eliminado: {archivo}")

def hay_que_revisar(only_boolean=False):
    def tiene_pendientes(archivo):
        with open(archivo, newline='', encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for fila in reader:
                if fila.get("No recibido", "").strip(): #fila.get("Transferencia?", "").strip(): # si tenia transferencias me hacia OCR
                    print(fila)
                    return True
        return False

    pendientes_ont = tiene_pendientes(archivo_ONT)
    pendientes_deco = tiene_pendientes(archivo_DECO)

    if only_boolean:
        return pendientes_ont or pendientes_deco

    hay_pendientes = False

    if pendientes_ont:
        print(f"⚠️ Aún hay equipos en '{archivo_ONT}' que requieren transferencia o no fueron recibidos.")
        hay_pendientes = True

    if pendientes_deco:
        print(f"⚠️ Aún hay equipos en '{archivo_DECO}' que requieren transferencia o no fueron recibidos.")
        hay_pendientes =

    if pendientes_ont or pendientes_deco:
      print("🚫 Revisar equipos no recibidos")

    return hay_pendientes

### **Funciones de comunicación con Google Sheets, Forms y Frappe**

* `abrirFotosNoRecibidos()`: Esta función, marcada para uso local y no en Colab, extrae las URLs de las fotos de los equipos que no se han recibido. Su propósito es preparar los enlaces para su posterior descarga o visualización.

* `enviarOTForms(ot_number)`: Envía un número de Orden de Trabajo (`ot_number`) a una aplicación web de Google Apps Script. Esto se utiliza para registrar o notificar la finalización de una OT.

* `ultimaOTconsumida()`: Lee la última fila del archivo `crudo.csv` para encontrar el número de la última Orden de Trabajo procesada y luego llama a `enviarOTForms()` para enviar este número a la aplicación de Google Forms.

* `agregar_comentario()`: Se conecta a la API de Frappe para añadir un comentario a un documento específico. Recibe el tipo y nombre del documento, así como el contenido del comentario, y maneja de manera robusta los errores de conexión o de la API.

In [ ]:
def abrirFotosNoRecibidos(): # sin usar si es en colab
    archivos = [archivo_ONT, archivo_DECO]
    campo_fotos = "foto (no recibido)"
    urls = set()

    for archivo in archivos:
        try:
            with open(archivo, newline='', encoding="utf-8") as f:
                reader = csv.DictReader(f)
                for fila in reader:
                    fotos_raw = fila.get(campo_fotos, "").strip()
                    if fotos_raw:
                        for link in fotos_raw.split("\n"):
                            url = link.strip()
                            if url.startswith("http"):
                                urls.add(url)
        except Exception as e:
            print(f"⚠️ No se pudo procesar {archivo}: {e}")

def enviarOTForms(ot_number):
    """
    Envía un número de OT a la Web App de Google Apps Script.

    Args:
        ot_number (str): El número de OT que se enviará.

    Returns:
        requests.Response: El objeto de respuesta de la petición.
    """
        # Datos que se enviarán en el cuerpo de la petición (formato JSON)
    payload = {
        "otNumber": ot_number
    }

    # Cabeceras para indicar que el contenido es JSON
    headers = {
        "Content-Type": "application/json"
    }

    try:
        # Realizar la petición POST
        response = requests.post(FormsAppscript, data=json.dumps(payload), headers=headers)

        # Verificar si la petición fue exitosa (código de estado 200)
        if response.status_code == 200:
            print(f"Enviado a Forms")
        else:
            print(f"Error en la petición. Código de estado: {response.status_code}")
            print(f"Respuesta del servidor: {response.text}")

        return response

    except requests.exceptions.RequestException as e:
        print(f"Se produjo un error al conectar con la Web App: {e}")
        return None

def ultimaOTconsumida():
    archivo = archivo_crudo
    ultima_fila = None
    with open(archivo, newline='', encoding="utf-8") as f:
        reader = csv.reader(f)
        for fila in reader:
            ultima_fila = fila
    if ultima_fila and len(ultima_fila) > 5:
        print()
        print()
        print(f"Última OT del {archivo}: {ultima_fila[5]}")
        enviarOTForms(ultima_fila[5])
        print()
        print()
    else:
        print(f"No se pudo leer la OT en la última fila de '{archivo}'.")




def agregar_comentario(doctype: str, docname: str, contenido: str, remitente: str = "aguida@qpsrl.com.ar") -> dict:
    """
    Agrega un comentario a un documento en Frappe utilizando las API Keys y endpoint predefinidos.

    Args:
        doctype (str): El tipo de documento de referencia en Frappe (ej: "Stock Entry", "Delivery Note").
        docname (str): El nombre o ID del documento en Frappe (ej: "MAT-STE-2025-01263").
        contenido (str): El contenido del comentario, se espera en formato HTML (ej: "<p>Mi comentario</p>").
        remitente (str, optional): Email del remitente. Por defecto es "aguida@qpsrl.com.ar".

    Returns:
        dict: Resultado de la operación, indicando éxito, errores y la respuesta de Frappe.
    """
    if not API_KEY or not API_SECRET:
        print("Error: API credentials (API_KEY or API_SECRET) are not set.")
        return {"success": False, "error": "API credentials not found"}

    if not FRAPPE_API_crudo:
        print("Error: Frappe API endpoint (FRAPPE_API_crudo) is not set. Check CSV fetch.")
        return {"success": False, "error": "Frappe API endpoint not found"}

    try:
        # Usar el método add_comment de Frappe para una integración más robusta
        url = f"{FRAPPE_API_crudo}/api/method/frappe.desk.form.utils.add_comment"

        payload = {
            "reference_doctype": doctype,
            "reference_name": docname,
            "content": contenido,
            "comment_email": remitente,
            "comment_by": remitente.split("@")[0] # Utiliza la parte antes del @ como nombre
        }

        headers = {
            "Authorization": f"token {API_KEY}:{API_SECRET}",
            "Content-Type": "application/json",
            "Accept": "application/json"
        }

        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # Lanza una excepción para errores HTTP (4xx o 5xx)

        response_data = response.json()

        print(f"🗨️ Comentario agregado a {docname} ({doctype}) usando add_comment")
        return {
            "success": True,
            "doctype": doctype,
            "docname": docname,
            "comment": contenido,
            "response": response_data
        }

    except requests.exceptions.RequestException as e:
        # Captura errores específicos de requests (conexión, HTTP, etc.)
        error_message = f"Request Error: {e}"
        if response.status_code:
            error_message += f" (HTTP {response.status_code}: {response.text})"
        print(f"❌ Error al enviar comentario a Frappe: {error_message}")
        return {
            "success": False,
            "error": error_message,
            "doctype": doctype,
            "docname": docname
        }
    except json.JSONDecodeError as e:
        # Captura errores si la respuesta no es JSON válido
        error_message = f"JSON Decode Error: {e}. Response text: {response.text}"
        print(f"❌ Error al procesar la respuesta JSON: {error_message}")
        return {
            "success": False,
            "error": error_message,
            "doctype": doctype,
            "docname": docname
        }
    except Exception as e:
        # Captura cualquier otro error inesperado
        print(f"❌ Error inesperado en agregar_comentario: {e}")
        return {
            "success": False,
            "error": str(e),
            "doctype": doctype,
            "docname": docname
        }

### **Funciones de obtención de datos y filtrado básico**

* `obtenerCrudo()`: Esta función se encarga de descargar los datos directamente de una planilla de Google Sheets. Filtra las filas que ya han sido procesadas, añade la etiqueta " - QPS" a los nombres de los técnicos si es necesario, y guarda los datos limpios en un archivo CSV local para su posterior uso. Si no encuentra filas pendientes, termina el script para evitar procesamientos innecesarios.

* `obtenerSeriesFrappe()`: Se comunica con la API de Frappe para obtener una lista de todos los números de serie (`Serial No`) que se encuentran en el sistema. Los guarda en un archivo CSV (`Series.csv`) para poder comparar y validar los datos de la planilla con el inventario del ERP.

* `obtenerProductos()`: Descarga un reporte de "Stock Projected Qty" desde la API de Frappe. Este reporte contiene información sobre la cantidad de artículos en cada almacén. La función procesa esta respuesta y guarda los datos en un archivo CSV (`productos.csv`), lo que es esencial para realizar verificaciones de stock antes de crear las notas de entrega.

In [49]:
def obtenerCrudo():
    print("Descargando CSV...")
    resp = requests.get(planilla_anual)
    resp.raise_for_status()

    lineas = resp.content.decode("utf-8").splitlines()

    print("Filtrando filas que no hayan sido consumidas...")
    reader = csv.reader(lineas, delimiter=',')
    filas_filtradas = []

    for fila in reader:
        if len(fila) > 25 and fila[4].strip() == "" and ( # acá en podes cambiar el de fila 4 a Si por si hay algun serie que quedó en el limbo
            fila[23].strip() == "EXITOSA" or
            fila[23].strip() == "CONTINGENCIA / SUSPENDIDA"
        ):
            # Columna D (índice 3) es el técnico. Le agregamos " - QPS" si no lo tiene.
            tecnico = fila[3].strip()
            if tecnico and not tecnico.endswith(" - QPS"):
                fila[3] = f"{tecnico} - QPS"
            filas_filtradas.append(fila)

    if not filas_filtradas:
        print("🚫 No hay filas pendientes de consumo. Terminando ejecución.")
        sys.exit(0)

    with open(archivo_crudo, "w", newline='', encoding="utf-8") as f_out:
        writer = csv.writer(f_out, delimiter=',')
        writer.writerows(filas_filtradas)

    print(f"✅ Filtrado completo. {len(filas_filtradas)} filas guardadas en '{archivo_crudo}'")


def obtenerSeriesFrappe():
    print("Consultando series en Frappe...")

    FRAPPE_API_serie = FRAPPE_API+"/Serial%20No"
    HEADERS = {
        "Authorization": f"token {API_KEY}:{API_SECRET}"
    }

    campos = [
        "name", "docstatus", "item_code", "item_name",
        "warehouse", "creation", "modified", "_user_tags"
    ]

    fields_param = json.dumps(campos)  # ✅ Convierte a JSON válido
    url = f"{FRAPPE_API_serie}?fields={fields_param}&limit_page_length=1000"

    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        data = response.json().get("data", [])

        with open(archivo_Series, "w", newline='', encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["sr"] + campos)  # encabezados

            for i, fila in enumerate(data, 1):
                writer.writerow([
                    i,
                    fila.get("name", ""),
                    fila.get("docstatus", ""),
                    fila.get("item_code", ""),
                    fila.get("item_name", ""),
                    fila.get("warehouse", ""),
                    fila.get("creation", ""),
                    fila.get("modified", ""),
                    fila.get("_user_tags", "")
                ])

        print(f"✅ {len(data)} series guardadas en '{archivo_Series}'")

    except requests.exceptions.RequestException as e:
        print(f"❌ Error al hacer la solicitud a Frappe: {e}")



def obtenerProductos():
    """Función principal para obtener el stock proyectado"""
    print("Iniciando descarga de stock proyectado...")

    # Configuración de headers que funciona
    headers = {
        "Authorization": f"token {API_KEY}:{API_SECRET}",
        "Content-Type": "application/x-www-form-urlencoded",
        "ngrok-skip-browser-warning": "true",
        "X-Frappe-CSRF-Token": "None"
    }

    # Payload que funciona (según pruebas)
    payload = {
        "cmd": "frappe.desk.query_report.run",
        "report_name": "Stock Projected Qty",
        "filters": json.dumps({"company": "Quality Plus", "ignore_permissions": 1}),
        "limit": "1000"
    }

    try:
        # Enviar solicitud
        response = requests.post(
            f"{FRAPPE_API}/api/method/frappe.desk.query_report.run",
            headers=headers,
            data=urlencode(payload)
        ) # Added the missing closing parenthesis here

        # Verificar respuesta
        response.raise_for_status()
        data = response.json()

        if not data.get("message") or not isinstance(data["message"].get("result"), list):
            raise Exception("La respuesta no contiene datos válidos")

        resultados = data["message"]["result"]
        print(f"Se obtuvieron {len(resultados)} registros de stock")

        # Columnas fijas basadas en el reporte Stock Projected Qty
        columnas = [
            "item_code", "item_name","warehouse", "actual_qty"
        ]

        # Guardar en CSV
        with open(archivo_productos, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=columnas, extrasaction='ignore')
            writer.writeheader()

            for row in resultados:
                # Asegurarnos de que sea un diccionario
                if isinstance(row, dict):
                    writer.writerow(row)
                else:
                    # Si es una lista, convertir a diccionario con las columnas conocidas
                    if isinstance(row, list) and len(row) == len(columnas):
                        writer.writerow(dict(zip(columnas, row)))
                    #else:
                    #    print(f"⚠️ Formato de fila no reconocido: {row}")

        print(f"✅ Datos guardados correctamente en {archivo_productos}")
        return True

    except requests.exceptions.RequestException as e:
        print(f"❌ Error en la conexión: {str(e)}")
        if hasattr(e, 'response') and e.response:
            print(f"Respuesta del servidor: {e.response.text}")
    except Exception as e:
        print(f"❌ Error al procesar los datos: {str(e)}")
        if 'data' in locals() and data.get('message') and data['message'].get('result'):
             print(f"Tipo de datos recibidos: {type(data['message']['result'][0])}")
        else:
             print("Tipo de datos recibidos: N/A")


    return False

### **Funciones de filtrado y categorización**

* `extraer_columnas_ont()`: Esta función lee los datos de la planilla (`crudo.csv`) y los de los números de serie (`series_frappe.csv`) para procesar los equipos ONT (modems de fibra). Filtra las filas de "instalación, cambio de domicilio" y determina si un equipo fue recibido, si requiere una transferencia de stock o si debe ser consumido en la nota de entrega. El resultado se guarda en `columnas_extraidasONT.csv`.

* `extraer_columnas_deco()`: Similar a la función anterior, pero se enfoca en los decodificadores. Procesa las filas del archivo `crudo.csv` que contienen series de Decos. Esta función también verifica el estado de cada Deco (si requiere transferencia, fue recibido o debe consumirse) y guarda la información en un archivo CSV específico para Decos (`columnas_extraidasDeco.csv`).

* `extraer_consumibles()`: Esta función es responsable de calcular el consumo de consumibles (como cables y conectores) basándose en los datos de las OTs y el stock disponible. Carga la información de stock desde `productos.csv` y aplica una lógica específica para ciertos ítems, ajustando el consumo solicitado con el stock real disponible. Finalmente, genera un informe detallado de los consumos finales en `reporte_consumibles.csv`.

In [50]:
def extraer_columnas_ont():
    archivo_entrada = archivo_crudo
    archivo_salida = archivo_ONT

    encabezados = [
        "ONT (modem fibra)",
        "foto",
        "Cargó OT",
        "Origen equipo",
        "Transferencia?",
        "No recibido",
        "foto (no recibido)",
        "Consumir"
    ]

    # Leer series_frappe.csv y construir un diccionario {serie: warehouse}
    dic_ont_origen = {}
    with open(archivo_Series, newline='', encoding="utf-8") as f_series:
        reader_series = csv.reader(f_series)
        next(reader_series)  # saltar encabezado
        for fila in reader_series:
            if len(fila) > 5:
                serie = fila[1].strip()
                warehouse = fila[5].strip()
                dic_ont_origen[serie] = warehouse

    # Leer crudo.csv y procesar
    with open(archivo_entrada, newline='', encoding="utf-8") as f_in:
        reader = csv.reader(f_in)
        filas_salida = []

        for fila in reader:
            if len(fila) <= 9:
                continue  # ignorar filas mal formateadas

            # Verificar si en columna Z (índice 26 - 1 = 25) está "INSTALACION CAMBIO DE DOMICILIO"
            if len(fila) > 25 and fila[25].strip().upper() == "INSTALACION CAMBIO DE DOMICILIO":
                continue  # saltar esta fila

            ont = fila[9].strip()
            if not ont:
                continue

            foto = fila[8].strip()
            cargo_ot_base = fila[3].strip()
            cargo_ot = f"{cargo_ot_base}" if cargo_ot_base else ""

            # Determinar origen equipo
            if ont in dic_ont_origen and dic_ont_origen[ont].strip():
                origen_equipo = dic_ont_origen[ont].strip()
            elif ont not in dic_ont_origen:
                origen_equipo = "No"
            else:
                origen_equipo = ""

            # Transferencia (solo si origen_equipo no es "No")
            if origen_equipo and origen_equipo != cargo_ot and origen_equipo != "No":
                transferencia = f"{origen_equipo}, {cargo_ot}"
            else:
                transferencia = ""

            no_recibido = ""
            foto_no_recibido = ""

            if origen_equipo == "No":
                no_recibido = ont
                foto_no_recibido = "\n".join([f.strip() for f in foto.split(",") if f.strip()])

            consumir = ""
            if origen_equipo != "No" and origen_equipo == cargo_ot:
                consumir = ont

            filas_salida.append([
                ont,
                foto,
                cargo_ot,
                origen_equipo,
                transferencia,
                no_recibido,
                foto_no_recibido,
                consumir
            ])

    with open(archivo_salida, "w", newline='', encoding="utf-8") as f_out:
        writer = csv.writer(f_out)
        writer.writerow(encabezados)
        writer.writerows(filas_salida)

    print(f"✅ Archivo '{archivo_salida}' generado con {len(filas_salida)} filas filtradas.")


def extraer_columnas_deco():
    archivo_entrada = archivo_crudo
    archivo_salida = archivo_DECO

    encabezados = [
        "Deco",
        "foto",
        "Cargó OT",
        "Origen equipo",
        "Transferencia?",
        "No recibido",
        "foto (no recibido)",
        "Consumir"
    ]

    # Leer series_frappe.csv y construir un diccionario {serie: warehouse}
    dic_deco_origen = {}
    with open(archivo_Series, newline='', encoding="utf-8") as f_series:
        reader_series = csv.reader(f_series)
        next(reader_series)  # saltar encabezado
        for fila in reader_series:
            if len(fila) > 5:
                serie = fila[1].strip()
                warehouse = fila[5].strip()
                dic_deco_origen[serie] = warehouse

    # Leer crudo.csv y procesar
    with open(archivo_entrada, newline='', encoding="utf-8") as f_in:
        reader = csv.reader(f_in)
        filas_salida = []

        for fila in reader:
            # ❗ Ignorar si columna AB (índice 27) es "INSTALACION CAMBIO DE DOMICILIO"
            if len(fila) > 27 and fila[27].strip().upper() == "INSTALACION CAMBIO DE DOMICILIO":
                continue

            if len(fila) > 10 and fila[10].strip() != "":
                deco_series = fila[10].strip().split()  # Puede haber múltiples series separados por espacio
                foto = fila[8].strip()
                cargo_ot_base = fila[3].strip()
                cargo_ot = f"{cargo_ot_base}" if cargo_ot_base else ""

                for deco in deco_series:
                    origen_equipo = "No" # esto es por si no se recibió el equipo
                    if deco in dic_deco_origen and dic_deco_origen[deco].strip():
                        origen_equipo = dic_deco_origen[deco].strip()
                    elif deco not in dic_deco_origen:
                        origen_equipo = "No"
                    else:
                        origen_equipo = ""

                    # Transferencia (solo si origen_equipo no es "No")
                    if origen_equipo and origen_equipo != cargo_ot and origen_equipo != "No":
                        transferencia = f"{origen_equipo}, {cargo_ot}"
                    else:
                        transferencia = ""

                    no_recibido = ""
                    foto_no_recibido = ""
                    if origen_equipo == "No":
                        no_recibido = deco
                        foto_no_recibido = "\n".join([f.strip() for f in foto.split(",") if f.strip()])

                    # Consumir: si no requiere transferencia y no tiene origen "No"
                    consumir = ""
                    if origen_equipo != "No": # and origen_equipo == cargo_ot: # si el equipo se recibió y el origen es el que lo cargó
                        consumir = deco

                    filas_salida.append([
                        deco,
                        foto,
                        cargo_ot,
                        origen_equipo,
                        transferencia,
                        no_recibido,
                        foto_no_recibido,
                        consumir
                    ])

    with open(archivo_salida, "w", newline='', encoding="utf-8") as f_out:
        writer = csv.writer(f_out)
        writer.writerow(encabezados)
        writer.writerows(filas_salida)

    print(f"✅ Archivo '{archivo_salida}' generado con {len(filas_salida)} filas.")

def extraer_consumibles():
    # Asegúrate de que las variables globales sean accesibles
    global archivo_consumibles, archivo_productos, archivo_crudo, archivo_DECO

    archivo_salida = archivo_consumibles
    item_code_fijo = "20710003" # Ejemplo: Cable UTP 1 metro
    item_code_extra1 = "110100372" # Ejemplo: Conector SC/APC
    item_code_extra2 = "320500065" # Ejemplo: Pigtail SC/APC

    # Ítems que se consumen "por completo según stock total" (independientemente de la cantidad solicitada)
    ITEMS_CONSUMIR_TODO_STOCK = {
        "140110033", "170200005", "170200010",
        "170300004", "20310001", "20710010"
    }

    print("🛠️ Iniciando extracción y cálculo de consumibles...")

    # 1. Cargar diccionario de productos y stock disponible
    # productos: item_code -> item_name
    # stock_disponible: item_code -> warehouse -> actual_qty (float)
    productos = {}
    stock_disponible = defaultdict(lambda: defaultdict(float))

    if os.path.exists(archivo_productos):
        with open(archivo_productos, newline='', encoding="utf-8") as f_stock:
            reader = csv.DictReader(f_stock)
            for row in reader:
                codigo = row.get("item_code", "").strip()
                nombre = row.get("item_name", "").strip()
                almacen = row.get("warehouse", "").strip()
                try:
                    stock = float(row.get("actual_qty", 0))
                except ValueError:
                    stock = 0.0
                    print(f"⚠️ Stock no numérico para {codigo} en {almacen}. Asumiendo 0.")

                if codigo and almacen: # Asegurarse de que tengamos un código y un almacén válidos
                    productos[codigo] = nombre
                    stock_disponible[codigo][almacen] = stock
    else:
        print(f"❌ Advertencia: Archivo '{archivo_productos}' no encontrado. No se aplicará control de stock para NINGÚN ítem.")
        # Si el archivo de stock no existe, todos los ítems se tratarán como si no tuvieran stock para control,
        # lo que significa que solo se consumirán si NO están en ITEMS_CONSUMIR_TODO_STOCK (y por lo tanto, su stock será 0).

    # 2. Contar ocurrencias por (item_code, warehouse) de lo que se NECESITA consumir
    # conteo_solicitado: item_code -> warehouse -> qty_solicitada (int)
    conteo_solicitado = defaultdict(lambda: defaultdict(int))

    # Procesar archivo_crudo (columna N: cables, columna O: cantidad fija)
    if os.path.exists(archivo_crudo):
        with open(archivo_crudo, newline='', encoding="utf-8") as f_crudo:
            reader = csv.reader(f_crudo)
            for fila in reader:
                if len(fila) < 15: # Necesitamos hasta la columna O (índice 14)
                    continue

                campo_cable = fila[13].strip() # Columna N
                campo_consumible = fila[14].strip() # Columna O
                cargo_ot_base = fila[3].strip() # Columna D (Técnico / Almacén)
                warehouse = cargo_ot_base if cargo_ot_base else "Default Warehouse"

                # 🔹 Cables (de columna N)
                if " - " in campo_cable:
                    item_code_cable = campo_cable.split(" - ")[0].strip()
                    if item_code_cable:
                        conteo_solicitado[item_code_cable][warehouse] += 1

                # 🔹 Cantidades para item_code fijo (de columna O)
                if campo_consumible:
                    try:
                        cantidad_fija = int(campo_consumible)
                        if cantidad_fija > 0:
                            conteo_solicitado[item_code_fijo][warehouse] += cantidad_fija
                    except ValueError:
                        print(f"⚠️ Valor no numérico en columna O de '{archivo_crudo}' para {warehouse}: '{campo_consumible}'. Ignorado.")
    else:
        print(f"❌ Advertencia: Archivo '{archivo_crudo}' no encontrado. No se procesarán consumibles del crudo.")

    # Procesar serializados desde archivo_DECO
    if os.path.exists(archivo_DECO):
        with open(archivo_DECO, newline='', encoding="utf-8") as f_deco:
            reader = csv.DictReader(f_deco)
            for fila in reader:
                serie = fila.get("Consumir", "").strip()
                almacen = fila.get("Cargó OT", "").strip()

                if serie and almacen:
                    warehouse_deco = almacen
                    conteo_solicitado[item_code_extra1][warehouse_deco] += 1
                    conteo_solicitado[item_code_extra2][warehouse_deco] += 2
    else:
        print(f"❌ Advertencia: Archivo '{archivo_DECO}' no encontrado. No se procesarán serializados de DECO.")

    # 3. Calcular consumos finales aplicando la lógica de stock
    # consumos_finales: item_code -> warehouse -> {'solicitada': X, 'real': Y}
    consumos_finales = defaultdict(lambda: defaultdict(lambda: {'solicitada': 0, 'real': 0}))

    # (Lógica original de procesamiento de ítems solicitados)
    for item_code, almacenes_solicitados in conteo_solicitado.items():
        for warehouse, qty_solicitada in almacenes_solicitados.items():
            qty_consumida_real = 0

            stock_actual_item_wh = stock_disponible.get(item_code, {}).get(warehouse, 0.0)
            if item_code not in productos and stock_disponible.get(item_code):
                # Se podría buscar el nombre en otro lado si se necesita,
                # pero por ahora no es crucial para la lógica de stock.
                pass

            if item_code in ITEMS_CONSUMIR_TODO_STOCK:
                # El problema es que esta lógica solo se ejecuta si el ítem está en `conteo_solicitado`.
                # La solución es mover el manejo de ITEMS_CONSUMIR_TODO_STOCK a una nueva sección
                # para que se procesen incluso si no fueron explícitamente solicitados.
                # Aquí, la lógica original está causando que solo se consuman si se solicitan.
                # Lo dejaré como estaba para mostrar que hay que cambiarlo
                qty_consumida_real = int(stock_actual_item_wh)
                # ... (los print originales) ...
            else:
                # El resto de los ítems (control estricto de solicitado vs. disponible)
                qty_consumida_real = min(qty_solicitada, int(stock_actual_item_wh))
                # ... (los print originales) ...

            consumos_finales[item_code][warehouse]['solicitada'] = qty_solicitada
            consumos_finales[item_code][warehouse]['real'] = qty_consumida_real

            stock_disponible[item_code][warehouse] -= qty_consumida_real
            if stock_disponible[item_code][warehouse] < 0.00001:
                stock_disponible[item_code][warehouse] = 0.0

    # --- manejar ITEMS_CONSUMIR_TODO_STOCK ---
    print("\n🔄 Procesando ítems de 'consumir todo el stock'...")
    for item_code_consumir in ITEMS_CONSUMIR_TODO_STOCK:
        # Solo procesar si el ítem existe en nuestro diccionario de stock
        if item_code_consumir in stock_disponible:
            for warehouse, stock_actual in stock_disponible[item_code_consumir].items():
                # Verifica que el almacén comience con "51"
                if warehouse.startswith("51"):
                    # Si no se solicitó (el valor 'real' es 0), lo actualizamos aquí.
                    if consumos_finales[item_code_consumir][warehouse]['real'] == 0 and stock_actual > 0:
                        qty_consumida_real = int(stock_actual)

                        # Registrar en consumos_finales
                        # La cantidad solicitada se deja en 0 ya que no hubo una solicitud explícita
                        consumos_finales[item_code_consumir][warehouse]['solicitada'] = 0
                        consumos_finales[item_code_consumir][warehouse]['real'] = qty_consumida_real

                        print(f"✅ Consumiendo {qty_consumida_real} de {item_code_consumir} en {warehouse} (ítem de consumo total).")

    # 4. Generar archivo de salida con los consumos finales
    filas_totales = 0
    with open(archivo_salida, "w", newline='', encoding="utf-8") as f_out:
        writer = csv.writer(f_out)
        writer.writerow(["item_code", "item_name", "warehouse", "qty_solicitada", "qty"])

        for item_code, almacenes_finales in consumos_finales.items():
            item_name = productos.get(item_code, f"❓ DESCONOCIDO ({item_code})")
            for warehouse, qtys in almacenes_finales.items():
                qty_solicitada_original = qtys['solicitada']
                qty_consumida_real = qtys['real']

                # Solo escribir si se solicitó o se consumió algo.
                if qty_solicitada_original > 0 or qty_consumida_real > 0:
                    writer.writerow([
                        item_code,
                        item_name,
                        warehouse,
                        qty_solicitada_original,
                        qty_consumida_real
                    ])
                    filas_totales += 1

    print(f"✅ Archivo '{archivo_salida}' generado con {filas_totales} filas (consumos ajustados por stock y lista ITEMS_CONSUMIR_TODO_STOCK).")
    print("✨ Proceso de extracción y cálculo de consumibles completado.")

### **Procesos previos a la Delivery Note**

* `transferirPendientes()`: Esta función revisa los equipos marcados para transferencia en los archivos CSV (`columnas_extraidasONT.csv` y `columnas_extraidasDeco.csv`). Si encuentra un equipo que necesita ser transferido de un técnico a otro, se comunica con la API de Frappe para crear un `Stock Entry` (movimiento de stock). Esto asegura que el stock esté en el almacén correcto antes de intentar generar la nota de entrega. También maneja posibles errores de la API e intenta una solución alternativa en caso de fallos.

In [52]:
def transferirPendientes():
    if hay_que_revisar():
        return

    # Cargar mapa {serie: item_code}
    def construir_diccionario_series():
        dic = {}
        with open("series_frappe.csv", newline='', encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                serie = row["name"].strip()
                item_code = row["item_code"].strip()
                if serie and item_code:
                    dic[serie] = item_code
        return dic

    HEADERS = {
        "Authorization": f"token {API_KEY}:{API_SECRET}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    archivos = [
        ("columnas_extraidasONT.csv", "ONT (modem fibra)"),
        ("columnas_extraidasDeco.csv", "Deco")
    ]
    url_frappe = FRAPPE_API + "/Stock%20Entry"

    # Construir mapa serie -> item_code
    serie_to_item = construir_diccionario_series()

    transferencias_por_tecnico = defaultdict(list)

    print("🚚 Procesando transferencias pendientes...")

    # Configurar sesión con reintentos básicos
    session = requests.Session()
    adapter = requests.adapters.HTTPAdapter(max_retries=3)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    for archivo, campo_serie in archivos:
        with open(archivo, newline='', encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for fila in reader:
                serie = fila.get(campo_serie, "").strip()
                transferencia = fila.get("Transferencia?", "").strip()

                if not serie or not transferencia or "," not in transferencia:
                    continue

                origen, destino = [x.strip() for x in transferencia.split(",", 1)]
                item_code = serie_to_item.get(serie)

                if not item_code:
                    print(f"⚠️ No se encontró item_code para serie '{serie}', omitiendo.")
                    continue

                try:
                    # Verificar estado actual de la serie
                    url_serie = f"{FRAPPE_API}/Serial%20No/{serie}"
                    response_serie = session.get(url_serie, headers=HEADERS, timeout=10)

                    if response_serie.status_code != 200:
                        print(f"⚠️ No se pudo verificar serie {serie}: {response_serie.status_code}")
                        continue

                    serie_data = response_serie.json().get("data", {})
                    if serie_data.get("warehouse") != origen:
                        print(f"⚠️ Serie {serie} no está en {origen}, está en {serie_data.get('warehouse')}")
                        continue

                    # Crear la transferencia con enfoque alternativo para bundles
                    stock_entry = {
                        "doctype": "Stock Entry",
                        "stock_entry_type": "Material Transfer",
                        "from_warehouse": origen,
                        "to_warehouse": destino,
                        "docstatus": 1,
                        "items": [{
                            "item_code": item_code,
                            "qty": 1,
                            "allow_zero_valuation_rate": 1,
                            "use_serial_batch_fields": 1,  # Evitar creación de nuevo bundle
                            "serial_no": serie
                        }]
                    }

                    # Primero intentar con serial_no
                    response = session.post(
                        url_frappe,
                        headers=HEADERS,
                        json=stock_entry,
                        timeout=30
                    )

                    if response.status_code == 200:
                        docname = response.json().get("data", {}).get("name")  # <- Capturamos el docname
                        print(f"✅ Transferido '{serie}' de '{origen}' a '{destino}'")
                        transferencias_por_tecnico[destino].append(f"{serie} de {origen}")
                    else:
                        error_msg = response.json().get("message", response.text)
                        print(f"❌ Error al transferir '{serie}': {response.status_code} - {error_msg}")

                        # Si falla, intentar sin serial_no
                        if "ya está creado" in error_msg:
                            print("  ⚠️ Intentando solución alternativa sin serial_no...")
                            del stock_entry["items"][0]["serial_no"]
                            response_alt = session.post(
                                url_frappe,
                                headers=HEADERS,
                                json=stock_entry,
                                timeout=30
                            )
                            if response_alt.status_code == 200:
                                print(f"  ✅ Solución alternativa exitosa para '{serie}'")
                                transferencias_por_tecnico[destino].append(f"{serie} de {origen}")
                            else:
                                print(f"  ❌ Falló solución alternativa: {response_alt.status_code} - {response_alt.text}")

                except requests.exceptions.RequestException as e:
                    print(f"❌ Error de conexión al transferir '{serie}': {str(e)}")
                except Exception as e:
                    print(f"❌ Error inesperado al transferir '{serie}': {str(e)}")
                agregar_comentario(
                  doctype="Stock Entry",
                  docname=docname, # Reemplaza con un docname real de tu Frappe
                  contenido="Transferencia realizada de forma automatica para una delivery note"
                )
    print("✅ Transferencias completadas.")
    return transferencias_por_tecnico

#transferirPendientes()

### **Santo Grial: Generación de la Nota de Entrega**

* `deliveryNote(transferencias_por_tecnico)`: Esta es la función principal que crea las notas de entrega en Frappe. Agrupa todos los equipos ONT, decodificadores y consumibles por técnico, y luego genera un `Delivery Note` para cada uno. La función también agrega un comentario en el documento de Frappe que contiene una tabla HTML con los detalles de las Órdenes de Trabajo y las transferencias de stock que se realizaron previamente.

In [53]:
def deliveryNote(transferencias_por_tecnico):
    if hay_que_revisar():
        #print("🚫 No se generó la nota de entrega. Revisá los archivos antes de continuar.")
        return

    FRAPPE_API_delivery = f"{FRAPPE_API}/Delivery%20Note" # Use FRAPPE_API
    HEADERS = {
        "Authorization": f"token {API_KEY}:{API_SECRET}",
        "Content-Type": "application/json"
    }

    cliente = "Instalado"
    archivos_consumir = [archivo_ONT, archivo_DECO]

    # Comentarios por técnico desde crudo
    def comentarios_en_tabla_html():
        comentarios = defaultdict(list)
        try:
            with open(archivo_crudo, newline='', encoding="utf-8") as f:
                reader = csv.reader(f)
                # Skip header if present (assuming first row is header in original script)
                # next(reader, None)
                for fila in reader:
                    if len(fila) >= 15:
                        almacen = fila[3].strip()
                        if not almacen:
                            continue
                        # Ensure cell content is HTML-safe if necessary (e.g., escape < > &)
                        fila_html = "".join(f"<td>{celda.strip()}</td>" for celda in fila[:15])
                        comentarios[almacen].append(f"<tr>{fila_html}</tr>")
        except FileNotFoundError:
            print(f"Error: Archivo no encontrado: {archivo_crudo}")
        return comentarios

    comentarios_por_almacen = comentarios_en_tabla_html()

    # Diccionario de series
    dic_series = {}
    try:
        with open(archivo_Series, newline='', encoding="utf-8") as f_series:
            reader = csv.reader(f_series)
            next(reader) # Skip header
            for fila in reader:
                if len(fila) > 5:
                    serie = fila[1].strip()
                    item_code = fila[3].strip()
                    item_name = fila[4].strip()
                    dic_series[serie] = {"item_code": item_code, "item_name": item_name}
    except FileNotFoundError:
        print(f"Error: Archivo no encontrado: {archivo_Series}")

    agrupados = defaultdict(lambda: defaultdict(lambda: {"qty": 0, "seriales": []}))
    seriales_usados = set()

    for archivo in archivos_consumir:
        try:
            with open(archivo, newline='', encoding="utf-8") as f:
                reader = csv.DictReader(f)
                for fila in reader:
                    serie = fila.get("Consumir", "").strip()
                    almacen = fila.get("Cargó OT", "").strip()
                    if not serie or not almacen:
                        continue
                    if serie in seriales_usados:
                        print(f"⚠️ Serie duplicada ignorada: {serie}")
                        continue
                    if serie not in dic_series:
                        print(f"⚠️ Serie no encontrada en {archivo_Series}: {serie}")
                        continue
                    info = dic_series[serie]
                    key = (info["item_code"], info["item_name"])
                    agrupados[almacen][key]["qty"] += 1
                    agrupados[almacen][key]["seriales"].append(serie)
                    seriales_usados.add(serie)
        except FileNotFoundError:
            print(f"Error: Archivo no encontrado: {archivo}")

    # Consumibles
    consumibles_por_almacen = defaultdict(list)
    try:
        with open(archivo_consumibles, newline='', encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                item_code = row["item_code"].strip()
                item_name = row["item_name"].strip()
                warehouse = row["warehouse"].strip()
                try:
                    qty = int(row["qty"])
                    if qty > 0:
                        consumibles_por_almacen[warehouse].append({
                            "item_code": item_code,
                            "item_name": item_name,
                            "qty": qty,
                            "warehouse": warehouse
                        })
                except ValueError:
                    print(f"⚠️ Cantidad inválida en consumibles: {row['qty']} para item {item_name}")
    except FileNotFoundError:
        print(f"Error: Archivo no encontrado: {archivo_consumibles}")


    # Crear delivery note por técnico
    todos_los_almacenes = set(agrupados.keys()) | set(consumibles_por_almacen.keys())
    for almacen in todos_los_almacenes:
        items = []

        for (item_code, item_name), datos in agrupados.get(almacen, {}).items():
            items.append({
                "item_code": item_code,
                "item_name": item_name,
                "qty": datos["qty"],
                "warehouse": almacen,
                "serial_no": "\n".join(datos["seriales"])
            })

        for prod in consumibles_por_almacen.get(almacen, []):
            items.append({
                "item_code": prod["item_code"],
                "item_name": prod["item_name"],
                "qty": prod["qty"],
                "warehouse": almacen
            })

        if not items:
            print(f"ℹ️ No hay ítems para crear Delivery Note para el almacén: {almacen}")
            continue

        payload = {
            "customer": cliente,
            "posting_date": str(date.today()),
            "set_warehouse": almacen,
            "docstatus": 1, # Validar DeliveryNote
            "items": items,
        }

        print(f"📦 Enviando nota de entrega para '{almacen}' con {len(items)} ítems...")
        response = requests.post(FRAPPE_API_delivery, headers=HEADERS, json=payload)

        if response.status_code == 200:
            docname = response.json()["data"]["name"]
            print(f"✅ Nota de entrega creada: {FRAPPE_API_crudo}/app/delivery-note/{docname}")

            # Paso 3: comentario con datos de crudo
            filas_html_list = comentarios_por_almacen.get(almacen, [])
            if filas_html_list:
                full_comment_html = "<h3>Detalle del Consumo (Crudo):</h3>"
                full_comment_html += "<table border='1' style='width:100%; border-collapse: collapse;'>"
                full_comment_html += "<thead><tr>" + "".join([f"<th>Col{i+1}</th>" for i in range(15)]) + "</tr></thead>" # Add a simple header for clarity
                full_comment_html += "<tbody>"
                full_comment_html += "".join(filas_html_list) # Join the list of <tr>...</tr> strings
                full_comment_html += "</tbody></table>"

                agregar_comentario(
                  doctype="Delivery Note",
                  docname=docname,
                  contenido=full_comment_html
                )

            # Paso 4: comentario con transferencias realizadas a este técnico
            transferencias_list = transferencias_por_tecnico.get(almacen)
            if transferencias_list:
                transfer_comment_html = "<h3>Transferencias Relacionadas:</h3>"
                transfer_comment_html += "<table border='1' style='width:100%; border-collapse: collapse;'>"
                transfer_comment_html += "<tbody>"
                transfer_comment_html += "🚚 ".join([f"<tr><td>{t}</td></tr>" for t in transferencias_list]) # Join the list of <tr>...</tr> strings
                transfer_comment_html += "</tbody></table>"

                agregar_comentario(
                  doctype="Delivery Note",
                  docname=docname,
                  contenido=transfer_comment_html
                )

        else:
            print(f"❌ Error al crear nota de entrega para {almacen}:")
            print(response.status_code, response.text)


## OCR

### **Descarga de imágenes**

* `extraer_file_id_google_drive(url)`: Esta función extrae el ID único de un archivo de Google Drive a partir de su URL. Es una función de utilidad que permite al script identificar la imagen que necesita descargar.
* `download_image_from_drive(file_id, dest_path)`: Utiliza el ID del archivo de Google Drive para descargar la imagen y la guarda en una ruta local. También realiza una verificación básica para asegurarse de que el archivo descargado sea realmente una imagen válida.
* `extraer_links_unicos_de_csvs(csvs, columna="foto (no recibido)")`: Recorre los archivos CSV de ONT y Decos para encontrar todas las URLs de las fotos de los equipos que no fueron recibidos. Devuelve una lista de URLs únicas y un diccionario que mapea cada URL con el número de serie correspondiente, para saber qué imagen pertenece a qué equipo.

In [54]:
def extraer_file_id_google_drive(url):
    for patron in [r'id=([a-zA-Z0-9_\-]+)', r'/d/([a-zA-Z0-9_\-]+)']:
        m = re.search(patron, url)
        if m:
            return m.group(1)
    return None

def download_image_from_drive(file_id, dest_path):
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    r = requests.get(url)
    if r.status_code == 200:
        with open(dest_path, 'wb') as f:
            f.write(r.content)

        # Validar si es imagen válida
        try:
            img = cv2.imread(dest_path)
            if img is None:
                print(f"⚠️ Archivo no es una imagen válida: {file_id}")
                os.remove(dest_path)
                return False
        except Exception as e:
            print(f"❌ Error leyendo imagen: {file_id} - {e}")
            os.remove(dest_path)
            return False

        print(f"📥 Imagen descargada en {dest_path}")
        return True

    print(f"❌ Error al descargar {file_id} (HTTP {r.status_code})")
    return False


def extraer_links_unicos_de_csvs(csvs, columna="foto (no recibido)"):
    enlaces_unicos = set()
    mapeo_url_a_serie = {}

    for path in csvs:
        with open(path, encoding="utf-8") as f:
            reader = csv.DictReader(f)
            headers = reader.fieldnames

            # Detectar columna de serie (puede variar entre ONT o Deco)
            posibles_series = ["serie", "ONT (modem fibra)", "Deco", "Consumir", "No recibido"]
            col_serie = next((c for c in posibles_series if c in headers), None)

            if not col_serie:
                continue

            for i, fila in enumerate(reader):
                origen = fila.get("Origen equipo", "").strip().lower()
                if origen != "no":
                    continue

                serie = fila.get(col_serie, "").strip().upper()
                raw_links = fila.get(columna, "")

                if not serie:
                    continue
                if not raw_links:
                    continue

                for url in raw_links.splitlines():
                    url = url.strip()
                    if url.startswith("http"):
                        enlaces_unicos.add(url)
                        mapeo_url_a_serie[url] = serie

    return list(enlaces_unicos), mapeo_url_a_serie

### **Rotación de las imagenes**

* `rotar_imagen_cv2(img, angle)`: Esta es una función de utilidad que rota una imagen en un ángulo específico (90, 180 o 270 grados) usando la biblioteca `cv2`.
* `asegurar_apaisado(img)`: Rota automáticamente una imagen a una orientación horizontal si es que esta se encuentra en una orientación vertical.
* `ocr_en_varias_rotaciones(img_path, ocr_detector, prefijos_validos)`: Esta es la función principal de OCR. Realiza un reconocimiento de texto en varias rotaciones de una imagen (0 y 180 grados) para asegurar una lectura correcta. Da prioridad a la rotación que encuentre un número de serie con uno de los `prefijos_validos`. Si no encuentra ningún prefijo válido, devuelve el resultado de la rotación que haya detectado más texto como un intento de respaldo.

In [55]:
def rotar_imagen_cv2(img, angle):
    if angle == 90 or angle == 270:
        return cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE if angle == 90 else cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif angle == 180:
        return cv2.rotate(img, cv2.ROTATE_180)
    return img

def asegurar_apaisado(img):
    h, w = img.shape[:2]
    return cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE) if h > w else img


def ocr_en_varias_rotaciones(img_path, ocr_detector, prefijos_validos):
    """
    Realiza OCR en varias rotaciones de una imagen, priorizando la rotación
    que contenga un texto con un prefijo válido.
    """
    img = asegurar_apaisado(cv2.imread(img_path))
    mejor_resultado = None
    mejor_resultado_con_prefijo = None
    max_textos = 0

    for angle in [0, 180]:
        print(f"\n🔄 Rotación {angle}°")
        rotada = rotar_imagen_cv2(img, angle)
        temp = "/tmp/temp_ocr.jpg"
        cv2.imwrite(temp, rotada)
        resultado = ocr_detector.predict(temp)
        os.remove(temp)

        if not resultado:
            continue

        textos = resultado[0].get('rec_texts', [])
        print("📌 Textos:", textos)

        # --- Nueva lógica de selección ---
        contiene_prefijo = False
        texto_con_prefijo = None
        for t in textos:
            texto_limpio = t.strip().upper()
            if any(texto_limpio.startswith(p) for p in prefijos_validos):
                contiene_prefijo = True
                texto_con_prefijo = t  # Guarda el texto completo que contiene el prefijo
                break

        # Si se encuentra un prefijo válido en esta rotación, la guardamos como la mejor.
        if contiene_prefijo:
            mejor_resultado_con_prefijo = resultado
            # No necesitamos seguir buscando, esta es la mejor opción.
            break

        # Si no hay prefijo válido, nos quedamos con la rotación que tenga más texto
        # para tener un respaldo en caso de que no haya coincidencias de prefijos en ninguna rotación.
        if len(textos) > max_textos:
            mejor_resultado = resultado
            max_textos = len(textos)

    # Si encontramos un resultado con un prefijo válido, lo retornamos
    if mejor_resultado_con_prefijo:
        print("✅ Se encontró una rotación con prefijo válido. Retornando el resultado.")
        return mejor_resultado_con_prefijo

    # Si no, retornamos el resultado que tenga la mayor cantidad de texto
    if mejor_resultado:
        print("ℹ️ No se encontraron prefijos válidos. Retornando el resultado con más texto.")
    else:
        print("❌ No se detectó ningún texto en ninguna rotación.")

    return mejor_resultado

### **Filtrado y comparación de series**

* `filtrar_numeros_serie(resultados)`: Esta función toma los resultados del OCR y los filtra para encontrar números de serie válidos. Busca cadenas de texto que comiencen con cualquiera de los `prefijos_validos` y se asegura de que la cadena resultante tenga al menos 6 caracteres y sea alfanumérica.
* `cargar_series_no_recibidas(csvs)`: Lee los archivos CSV y extrae una lista de tuplas, donde cada tupla contiene el número de serie original y la URL de la foto del equipo que no fue recibido. Esto prepara los datos para el proceso de corrección.
* `procesar_fotos_con_ocr(...)`: Esta es la función principal que coordina el proceso de OCR. Primero, consulta la API de Frappe para obtener la lista completa de números de serie válidos. Luego, descarga las imágenes de los equipos no recibidos y ejecuta el OCR en cada una. Finalmente, compara las series detectadas por el OCR con la base de datos de Frappe para validar las correcciones y las exporta a un archivo CSV.
* `run_ocr_si_hay_que_revisar(...)`: Actúa como la función de entrada para el proceso de OCR. Llama a `extraer_links_unicos_de_csvs` para obtener las URLs de las fotos de los equipos no recibidos y luego a `procesar_fotos_con_ocr` para ejecutar el reconocimiento.
* `exportar_series_detectadas_csv(...)`: Genera un archivo CSV que mapea cada serie original (la que no fue recibida) con la serie corregida que fue detectada y validada por el OCR, facilitando la corrección manual o automática.
* `obtener_mapeo_file_id_a_idx(no_recibidos)` y `exportar_url_serie_a_csv(...)`: Funciones de utilidad para manejar la relación entre los IDs de archivo, las URLs y los números de serie originales.
* `comparar_series_viejos_y_nuevos(...)`: Compara las series originales con las series corregidas del OCR para identificar posibles errores. Utiliza el algoritmo de distancia de Levenshtein para sugerir las correcciones más probables, lo que ayuda a los usuarios a validar los resultados del OCR.

In [57]:
def filtrar_numeros_serie(resultados):
    """
    Filtra los números de serie de los resultados del OCR.
    La lógica se basa en buscar prefijos válidos dentro de la cadena de texto
    y extraer la parte del texto que comienza con ese prefijo.

    Args:
        resultados (list): Una lista de resultados de OCR, donde cada elemento es un diccionario
                         con una clave 'rec_texts' que contiene una lista de strings.

    Returns:
        list: Una lista de strings únicos que son números de serie válidos.
    """
    encontrados = set()

    for bloque in resultados:
        for txt in bloque.get('rec_texts', []):
            t = txt.strip().upper()

            for prefijo in prefijos_validos:
                if prefijo in t:
                    inicio = t.find(prefijo)

                    numero_serie_candidato = t[inicio:]

                    if len(numero_serie_candidato) >= 6 and numero_serie_candidato.isalnum():
                        encontrados.add(numero_serie_candidato)
                        break

    return list(encontrados)


def cargar_series_no_recibidas(csvs):
    """
    Retorna una lista de tuplas: (serie_original, link_individual)
    Si hay más de un link en la celda, los divide correctamente.
    """
    pares = []
    for path in csvs:
        with open(path, encoding="utf-8") as f:
            for fila in csv.DictReader(f):
                serie = fila.get("serie", "").strip().upper()
                enlaces = fila.get("foto (no recibido)", "").strip()
                if not serie or not enlaces:
                    continue
                for link in enlaces.splitlines():
                    if link.strip().startswith("http"):
                        pares.append((serie, link.strip()))
    return pares


def procesar_fotos_con_ocr(enlaces, ocr_detector, api_key, api_secret, frappe_api, no_recibidos=None):
    print("\n🔌 Obteniendo series del ERP...")
    try:
        headers = {"Authorization": f"token {api_key}:{api_secret}"}
        url = f"{frappe_api}/Serial%20No?fields=[\"name\"]&limit_page_length=1000"
        data = requests.get(url, headers=headers).json().get("data", [])
        series_erp = set(x["name"].upper() for x in data if x.get("name"))
        print(f"✅ {len(series_erp)} series obtenidas")
    except Exception as e:
        print(f"❌ Error ERP: {e}")
        return {}

    resultados, mapeos = {}, []
    filas_validas = []

    for i, url in enumerate(enlaces):
        print(f"\n📷 Imagen {i+1}/{len(enlaces)}: {url}")
        file_id = extraer_file_id_google_drive(url)
        if not file_id:
            print("⚠️ No se pudo extraer ID")
            continue

        path = f"/tmp/imagen_{i}.jpg"

        if not download_image_from_drive(file_id, path):
            continue

        ocr = ocr_en_varias_rotaciones(path, ocr_detector, prefijos_validos)
        series_detectadas = filtrar_numeros_serie(ocr or [])
        verificacion = {s: s in series_erp for s in series_detectadas}

        print("🔎 Series detectadas:", series_detectadas)
        print("✅ Validación:", verificacion)

        for s in series_detectadas:
            if verificacion.get(s):
                filas_validas.append((url, s))

        if no_recibidos:
            for detectada in series_detectadas:
                for original, link in no_recibidos.items():
                    if file_id in link and detectada != original:
                        mapeos.append(f"{original} -> {detectada}")

        resultados[url] = {
            "series_detectadas": series_detectadas,
            "verificacion": verificacion
        }

        os.remove(path)

    if filas_validas:
        with open(path_nuevos, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["url", "serie"])
            writer.writerows(filas_validas)
        print(f"\n✅ CSV generado: series_validas_detectadas.csv ({len(filas_validas)} filas)")
    else:
        print("\n⚠️ No se detectaron series válidas, no se generó CSV.")

    return {
        "resultados": resultados,
        "mapeos_sugeridos": mapeos
    }


def run_ocr_si_hay_que_revisar(
    ocr_detector,
    api_key,
    api_secret,
    frappe_api,
    archivo_ONT,
    archivo_DECO
):
    print("⚠️ Ejecutando OCR por no recibidos...")
    enlaces = extraer_links_unicos_de_csvs([archivo_ONT, archivo_DECO])[0]
    print(f"🔗 {len(enlaces)} enlaces únicos")

    no_recibidos = cargar_series_no_recibidas([archivo_ONT, archivo_DECO])
    resultado = procesar_fotos_con_ocr(enlaces, ocr_detector, api_key, api_secret, frappe_api, no_recibidos)

    for mapeo in resultado["mapeos_sugeridos"]:
        print("🔁", mapeo)

    return resultado

def exportar_series_detectadas_csv(
    resultados_ocr: dict,
    url_a_serie_original: dict,
    path_salida="series_detectadas_con_validacion.csv"
):
    """
    Exporta un CSV con:
    - serie original (de los no recibidos)
    - serie corregida detectada y validada por OCR

    Usa la URL como clave para relacionar datos.
    """

    filas = []

    for url, serie_original in url_a_serie_original.items():
        resultado = resultados_ocr.get(url)
        if not resultado:
            continue
        if "verificacion" not in resultado:
            continue

        # Filtrar solo series validadas True
        series_validas = [s for s, ok in resultado["verificacion"].items() if ok]

        # Si hay series válidas, hacer una fila por cada una
        for serie_corregida in series_validas:
            filas.append([serie_original, serie_corregida])

    # Guardar CSV con SOLO dos columnas: serie original y serie corregida
    with open(path_salida, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['serie_original', 'serie_corregida_valida'])
        writer.writerows(filas)

    print(f"✅ CSV generado: {path_salida} ({len(filas)} filas)")



def obtener_mapeo_file_id_a_idx(no_recibidos):
    """
    Devuelve un dict: file_id => índice en la lista de no_recibidos
    """
    mapeo = {}
    for idx, (_, url) in enumerate(no_recibidos):
        file_id = extraer_file_id_google_drive(url)
        if file_id:
            mapeo[file_id] = idx
    return mapeo

def exportar_url_serie_a_csv(url_a_serie_dict, path_salida=path_viejos):
    """
    Exporta un CSV con dos columnas:
    url, serie
    a partir de un diccionario {url: serie}
    """
    filas = [(url, serie) for url, serie in url_a_serie_dict.items()]

    with open(path_salida, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['url', 'serie'])
        writer.writerows(filas)

    print(f"✅ CSV generado: {path_salida} ({len(filas)} filas)")


def comparar_series_viejos_y_nuevos(
    path_viejos=path_viejos,
    path_nuevos=path_nuevos
):
    # Verificar existencia de archivos
    if not os.path.exists(path_viejos):
        print(f"❌ Archivo no encontrado: {path_viejos}")
        return
    elif not os.path.exists(path_nuevos):
        print(f"❌ Archivo no encontrado: {path_nuevos}")
        return

    def leer_csv(path):
        d = {}
        with open(path, encoding="utf-8") as f:
            for row in csv.DictReader(f):
                url = row["url"].strip()
                serie = row["serie"].strip().upper()
                d.setdefault(url, []).append(serie)
        return d

    def levenshtein(a, b):
        if len(a) < len(b):
            return levenshtein(b, a)
        if len(b) == 0:
            return len(a)
        previous_row = range(len(b) + 1)
        for i, c1 in enumerate(a):
            current_row = [i + 1]
            for j, c2 in enumerate(b):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row
        return previous_row[-1]

    def sugerir_correccion(serie_vieja, series_validas):
        if not series_validas:
            return ""
        similitudes = [(s, levenshtein(serie_vieja, s)) for s in series_validas]
        similitudes.sort(key=lambda x: x[1])
        return similitudes[0][0]  # menor distancia

    viejos = leer_csv(path_viejos)
    nuevos = leer_csv(path_nuevos)
    todas_las_urls = sorted(set(viejos) | set(nuevos))

    print("\n📊 Comparación de series:")
    for url in todas_las_urls:
        viejas = viejos.get(url, [])
        nuevas = nuevos.get(url, [])
        print(f"{url} - {viejas} -> {nuevas}")

    print("\n🔍 Sugerencias automáticas de corrección:")
    for url in todas_las_urls:
        viejas = viejos.get(url, [])
        nuevas = nuevos.get(url, [])
        for vieja in viejas:
            sugerida = sugerir_correccion(vieja, nuevas)
            if sugerida and sugerida != vieja:
                print(f"🔁 {url} - Corregir '{vieja}' -> '{sugerida}'")


## **Función principal (`main`)**

`main()`: Esta es la función principal que orquesta todo el flujo de trabajo del script. Ejecuta las siguientes etapas de forma secuencial:
1.  **Limpieza inicial**: Elimina archivos temporales de ejecuciones anteriores.
2.  **Obtención de datos**: Llama a las funciones para descargar la planilla de consumo y los datos de stock y series de Frappe.
3.  **Filtrado y procesamiento**: Procesa los datos para extraer la información de equipos ONT, decodificadores y consumibles.
4.  **Transferencias**: Revisa si hay equipos que necesitan ser transferidos entre técnicos y realiza las transferencias necesarias en el ERP.
5.  **Generación de notas de entrega**: Crea las notas de entrega (`Delivery Notes`) en Frappe, agrupando los ítems por técnico y agregando comentarios detallados.
6.  **OCR condicional**: Si aún quedan equipos marcados como "no recibidos", inicia un proceso de OCR (Reconocimiento Óptico de Caracteres). Este subproceso instala las librerías necesarias, descarga las fotos, las procesa para leer los números de serie, los valida con el ERP y genera un reporte de correcciones sugeridas.
7.  **Limpieza final**: Después de completar el flujo principal o el OCR, limpia los archivos CSV temporales.

In [58]:
def main():

    #os.system("clear") #quitar comentario en caso de usar en local, para mas estetica

    # Limpieza previa
    borrar_archivos_csv() #quitar comentario en caso de usar en local

    # Obtención de datos
    obtenerCrudo()
    obtenerSeriesFrappe()
    obtenerProductos()

    # Procesamiento
    extraer_columnas_ont()
    extraer_columnas_deco()
    extraer_consumibles()

    # Transferencias
    transferencias_por_tecnico = transferirPendientes()

    # Si tiene seguro que abre, sino no
    #abrirFotosNoRecibidos() # en el caso de colab no hace falta jsajdsja

    # Deliverys
    print("\n\n--- Generando Delivery Note ---")
    deliveryNote(transferencias_por_tecnico)

    # Ahora chequeo si hay que revisar con OCR:
    if hay_que_revisar(only_boolean=True):
        print("\n--- Revisando dependencias para comenzar con el OCR ---")

        """OCR batch desde enlaces de Google Drive con rotación automática"""
        # Instalaciones de dependencias de ocr(solo en Colab)
        # solo si hace falta
        !pip install pytesseract
        !apt-get install -y tesseract-ocr libtesseract-dev
        if gpu_o_cpu=="C" or gpu_o_cpu == "":
          !pip install paddlepaddle -f https://www.paddlepaddle.org.cn/whl/quick_install.html # en caso de usar local para poder usar el procesamiento a través de CPU y no de grafica, tarda mas solamente
        if gpu_o_cpu.upper() == "G":
          !pip install paddlepaddle-gpu==3.0.0 -i https://www.paddlepaddle.org.cn/packages/stable/cu118/
        !pip install paddleocr

        from paddleocr import PaddleOCR

        print("\n--- Lanzando OCR por equipos no recibidos ---")

        ocr_detector = PaddleOCR(det_model_dir=None, use_textline_orientation=True, lang='en')

        resultado_ocr = run_ocr_si_hay_que_revisar(
            ocr_detector,
            API_KEY,
            API_SECRET,
            FRAPPE_API,
            archivo_ONT,
            archivo_DECO
        )

        # Cargar no_recibidos como lista de tuplas (serie_original, url)
        no_recibidos = cargar_series_no_recibidas([archivo_ONT, archivo_DECO])

        # Convertir a dict {url: serie_original}
        url_a_serie_original = {url: serie for serie, url in no_recibidos}

        # series mal
        url_a_serie = extraer_links_unicos_de_csvs([archivo_ONT, archivo_DECO])[1]
        exportar_url_serie_a_csv(url_a_serie)

        comparar_series_viejos_y_nuevos()
        borrar_archivos_csv()

    else:
        print("\n--- Todos los equipos recibidos, seguimos con flujo normal ---")
        ultimaOTconsumida()
        borrar_archivos_csv()  # sacar el comentario en caso de usar local para que se puedan actualizar los archivos descargados

main()

🧹 Archivo eliminado: crudo.csv
🧹 Archivo eliminado: series_frappe.csv
🧹 Archivo eliminado: productos.csv
🧹 Archivo eliminado: columnas_extraidasONT.csv
🧹 Archivo eliminado: columnas_extraidasDeco.csv
🧹 Archivo eliminado: reporte_consumibles.csv
Descargando CSV...
Filtrando filas que no hayan sido consumidas...
✅ Filtrado completo. 30 filas guardadas en 'crudo.csv'
Consultando series en Frappe...
✅ 1000 series guardadas en 'series_frappe.csv'
Iniciando descarga de stock proyectado...
Se obtuvieron 104 registros de stock
✅ Datos guardados correctamente en productos.csv
✅ Archivo 'columnas_extraidasONT.csv' generado con 21 filas filtradas.
✅ Archivo 'columnas_extraidasDeco.csv' generado con 19 filas.
🛠️ Iniciando extracción y cálculo de consumibles...
⚠️ Valor no numérico en columna O de 'crudo.csv' para 51ROD636 (Pat. AD833WB) - QPS: '--------SIN UTILIZAR--------'. Ignorado.
⚠️ Valor no numérico en columna O de 'crudo.csv' para 51ROD637 (Pat. AC774WF) - QPS: '--------SIN UTILIZAR--------